# Data Cleaning and Pre Processing

### Importing relevant libraries

In [4]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
from bs4 import BeautifulSoup
import urllib3
print('Libraries imported.')

Libraries imported.


### Using BeautifulSoup to scrape the webpage for the relevant data, in this case the table of postcodes


In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source)

#Extracting Column Names
table_head = soup.find('tr')
header=table_head.text.split('\n')
header=header[1:4]


### Extracting data

In [6]:
data=soup.find('tbody').text.strip().split('\n')
dataf = [data[x:x+5] for x in range(0, len(data),5)]
df=pd.DataFrame(dataf)
df.head()
df=df.drop(df.index[0])
df=df.drop([3,4], axis=1)
df.columns=header
df.head()
df = df[df['Borough']!='Not assigned']
dfn=df

### Arranging data as we need, removing not assigned boroughs 

In [7]:
df = df[df['Borough']!='Not assigned']
df.loc[df['Neighbourhood']=='Not assigned', 'Neighbourhood'] = df[df['Neighbourhood']=='Not assigned']['Borough']

for postcode,g in df.groupby('Postcode'):
    df.loc[df['Postcode']==postcode, 'Neighbourhoods'] = ','.join(g['Neighbourhood'].tolist())
   

 
df=df[['Postcode','Borough','Neighbourhoods']]



df.drop_duplicates('Postcode',keep='first',inplace=True)
df.reset_index(drop=True)

,Postcode,Borough,Neighbourhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


### Loading geospatial coordinates and combining above dataframe and coordinates

In [8]:
geodf=pd.read_csv('Geospatial_Coordinates.csv')
geodf.shape
geodf=geodf.sort_values(by='Postal Code')


tst=df.sort_values(by='Postcode')
tst.loc(axis=1)['Latitude']=geodf[['Latitude']].values
tst.loc(axis=1)['Longitude']=geodf[['Longitude']].values

#to be used later in clustering
dft=tst
dft.to_csv('dft',index=None)
#final dataframe
tst=tst.reset_index(drop=True)
tst.to_csv('postcodes',index=None)

In [9]:
dft

,Postcode,Borough,Neighbourhoods,Latitude,Longitude
11,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
27,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
42,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
53,M1G,Scarborough,Woburn,43.770992,-79.216917
62,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
76,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
91,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
107,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
123,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
140,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
